# Testing

This script compares the human v. machine coding of recommendations.

In [19]:
import os
import re
import csv
import sys
import csv
import random
import ast

csv.field_size_limit(sys.maxsize)

9223372036854775807

## 1. Load

In [38]:
# load data
recs = []
with open("../Data/all-data-cown.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'from_COW': '690',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'to_COW': '615',
 'year': '2008'}

In [60]:
# load Erin
erin = []
with open("testing-3/erin-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                erin.append(row)
            
erin[1]

{'From': 'Tunisia',
 'action': "['continue']",
 'decision': 'consider',
 'id': '690',
 'institutions': '[]',
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': "['children']",
 'to': 'Romania',
 'year': '2008'}

In [61]:
# making literal lists
for i in erin:
    try:
        i['theme'] = ast.literal_eval(i['theme'])
        i['institutions'] = ast.literal_eval(i['institutions'])
        i['action'] = ast.literal_eval(i['action'])
    except Exception,e:
        print i
erin[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'id': '690',
 'institutions': [],
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': ['children'],
 'to': 'Romania',
 'year': '2008'}

In [62]:
# strip whitespace:
for i in erin:
    i['theme'] = [x.strip() for x in i['theme']]
    i['institutions'] = [x.strip() for x in i['institutions']]
    i['action'] = [x.strip() for x in i['action']]

In [63]:
# get ids
ids = [rec['id'] for rec in erin]

In [64]:
# get original sample
sample = []
for i in recs:
    if i['id'] in ids:
        sample.append(i)
print len(sample), sample[0]  

100 {'from_COW': '70', 'From': 'Mexico', 'to_COW': '140', 'text': '83.8 Enhance access to justice as well as improve the judicial system (Mexico);', 'decision': 'support', 'id': '92', 'to': 'Brazil', 'theme': "['judiciary']", 'notes': '', 'year': '2008', 'action': '[]', 'institutions': '[]'}


In [65]:
# make literal
for i in sample:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

sample[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'from_COW': '616',
 'id': '690',
 'institutions': [],
 'notes': '',
 'text': "58.14 To continue to work on improving the situation of children's rights (Algeria, Tunisia);",
 'theme': ['children'],
 'to': 'Romania',
 'to_COW': '360',
 'year': '2008'}

## Compare

In [66]:
# zip
matching = []
for i in range(100):
    dic = erin[i].copy()
    
    # Theme
    dic['theme-erin'] = erin[i]['theme']
    dic['theme-rochelle'] = sample[i]['theme']
    del dic['theme']
    
    if set(dic['theme-erin']) == set(dic['theme-rochelle']):
        dic['theme-match'] = True
    elif set(dic['theme-erin']).intersection(dic['theme-rochelle']):
        dic['theme-match'] = "Partial"
    else:
        dic['theme-match'] = False
    
    # institutions
    dic['institution-erin'] = erin[i]['institutions']
    dic['institution-rochelle'] = sample[i]['institutions']
    del dic['institutions']
    
    if set(dic['institution-erin']) == set(dic['institution-rochelle']):
        dic['institution-match'] = True
    elif set(dic['institution-erin']).intersection(dic['institution-rochelle']):
        dic['institution-match'] = "Partial"
    else:
        dic['institution-match'] = False
    
    # actions
    dic['action-erin'] = erin[i]['action']
    dic['action-rochelle'] = sample[i]['action']
    del dic['action']
    
    if set(dic['action-erin']) == set(dic['action-rochelle']):
        dic['action-match'] = True
    elif set(dic['action-erin']).intersection(dic['action-rochelle']):
        dic['action-match'] = "Partial"
    else:
        dic['action-match'] = False
    
    matching.append(dic)

In [67]:
# make lists of matches, etc
theme_match = [i for i in matching if i['theme-match'] == True]
theme_partial = [i for i in matching if i['theme-match'] == "Partial"]
theme_false = [i for i in matching if i['theme-match'] == False]

institution_match = [i for i in matching if i['institution-match'] == True]
institution_partial = [i for i in matching if i['institution-match'] == "Partial"]
institution_false = [i for i in matching if i['institution-match'] == False]

action_match = [i for i in matching if i['action-match'] == True]
action_partial = [i for i in matching if i['action-match'] == "Partial"]
action_false = [i for i in matching if i['action-match'] == False]

In [68]:
print "Theme" + "\n"
print "Match: " + str(len(theme_match))
print "Partial: " + str(len(theme_partial))
print "False: " + str(len(theme_false))
print "\n"

print "Institutions" + "\n"
print "Match: " + str(len(institution_match))
print "Partial: " + str(len(institution_partial))
print "False: " + str(len(institution_false))
print "\n"

print "Action" + "\n"
print "Match: " + str(len(action_match))
print "Partial: " + str(len(action_partial))
print "False: " + str(len(action_false))
print "\n"

Theme

Match: 74
Partial: 20
False: 6


Institutions

Match: 90
Partial: 4
False: 6


Action

Match: 78
Partial: 10
False: 12




In [70]:
for i in action_partial:
    print i['id'] + ": " + i['text']
    print "ERIN: " + str(i['action-erin'])
    print "ROCHELLE: " + str(i['action-rochelle']) + "\n"

4385: 57.8.1 and take all appropriate measures, in the fields of legislation, implementation and awareness-raising, to tackle domestic violence against women and children (Italy);
ERIN: ['take-measures']
ROCHELLE: ['implement', 'take-measures']

9370: 101.12. Continue to implement the core elements of the Children�s Act, which is a great step forwards in the achievement of the Millennium Development Goals (Angola);
ERIN: ['continue']
ROCHELLE: ['continue', 'implement']

10098: 96.16. Continue to improve access to basic services, such as health and education, with the full support and cooperation of the international community (Bangladesh);
ERIN: ['continue']
ROCHELLE: ['continue', 'cooperate', 'assistance']

12177: 106.12. Continue commitment to human rights through the ratification of the International Convention on the Protection of the Rights of All Migrant Workers and Members of Their Families in accordance with recommendation 1737 of 17 March 2006, adopted by the Parliamentary Ass

## 3. Transform to Binary SpreadSheet

In [ ]:
institutions = ['core', 'CRPD', 'ICCPR', 'eu-vaw', 'geneva', 'ICC', 'ILO', 'ICESR', 'achr', 'CAT', \
                'CRC', 'echr', '1954-stateless', 'traffick', 'CTOC', 'ICMW', 'ICERD', 'eu-child', 'sp-pr', \
                'PP', 'kampala', '1951refugees', 'pol-women', 'CPED', 'uncac', 'ottawa', '1961-stateless', 'CEDAW', \
                'unesco-ed', 'hague', 'genocide', 'HRC', 'unesco-expr', 'int-cord', 'frame-minorities', 'ind169',\
                'OP', 'ic-mercenaries', 'eu-migrant', 'un-indig', 'vienna', 'ouagadougou', 'UNHCR']

In [107]:
actions = ['accede', 'ratify', 'take-measures', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', \
           'implement', 'establish']

In [108]:
theme = ['death-pen', 'infrastructure ', 'intelligence', 'prisoners', 'elderly', 'culture', 'education', \
         'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity',\
         'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', \
         'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'civil-conflict', 'race', 'torture', 'food',\
         'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', \
         'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', \
         'military', 'property']

In [109]:
erin[0].keys()

['From',
 'sentiment',
 'text',
 'decision',
 'id',
 'to',
 'theme',
 'notes',
 'year',
 'action',
 'institutions']

In [110]:
for i in sample:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0 
    
    del i['to']
    del i['From']
    del i['text']
    del i['notes']
    del i['action']
    del i['theme']
    del i['decision']
    del i['institutions']
    del i['year']

In [111]:
for i in erin:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  
            
    del i['to']
    del i['From']
    del i['text']
    del i['year']
    del i['institutions']
    del i['action']
    del i['theme']
    del i['decision']
    del i['sentiment']

In [112]:
## writing sample

#writing column headings
keys = sample[0].keys()

#writing the rest
with open('testing-3/sample-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [113]:
## writing erin

#writing column headings
keys = erin[0].keys()

#writing the rest
with open('testing-3/erin-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(erin)